## Ансамблевая классификация
Исходный код: https://github.com/ellkrauze/decision_tree_classification
## Задание
1. Разработайте программу, которая выполняет классификацию заданного набора данных с помощью одной из техник ансамблевой классификации. Параметрами программы являются набор данных, ансамблевая техника (бэггинг, случайный лес или бустинг), количество участников ансамбля, а также параметры в соответствии с выбранной техникой ансамблевой классификации.
2. Проведите эксперименты на наборах данных из задания Классификация с помощью дерева решений, варьируя количество участников ансамбля (от 50 до 100 с шагом 10).
3. Выполните визуализацию полученных результатов в виде следующих диаграмм:
показатели качества классификации в зависимости от количества участников ансамбля для заданного набора данных; нанесите на диаграмму соответствующие значения, полученные в задании Классификация с помощью дерева решений.
## Теория
**Целью ансамблевых методов** является объединение прогнозов нескольких базовых оценок, построенных с использованием заданного алгоритма обучения, для улучшения обобщаемости/надежности по сравнению с одной оценкой.

Обычно выделяют два семейства ансамблевых методов:

1. В **методах усреднения** основным принципом является независимое построение нескольких оценщиков, а затем усреднение их прогнозов. В среднем комбинированная оценка обычно лучше, чем любая из оценок с одной базой, потому что ее дисперсия уменьшается.
Примеры: Bagging methods, Forests of randomized trees…

2. Напротив, в **методах бустинга** базовые оценки строятся последовательно, и каждый пытается уменьшить смещение комбинированной оценки. Мотивация состоит в том, чтобы объединить несколько слабых моделей для создания мощного ансамбля.
Примеры: AdaBoost, Gradient Tree Boosting…

In [42]:
import os
import pandas as pd
import category_encoders as ce
import sklearn.metrics 

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Split train and test data 
# based on given test size
# Parameters:
# - df is given dataset
# - class_col is the label column
# - test_size is the proportion of the dataset to include in the train split
def get_splitted_train_test(df, class_col, test_size):
    # Declare feature vector and target variable 
    X = df.drop([class_col], axis=1)
    y = df[class_col]
    # Split data into separate training and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 42)
    return (X_train, X_test, y_train, y_test)

In [3]:
def get_encoded_X_data(X_train, X_test):
    # encode variables with ordinal encoding
    cols = X_train.columns.values.tolist()
    encoder = ce.OrdinalEncoder(cols=cols)
    X_train = encoder.fit_transform(X_train)
    X_test = encoder.transform(X_test)
    return (X_train, X_test)

In [10]:
def evaluate(model, X_train, X_test, y_train, y_test):
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    print("TRAINIG RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True, zero_division=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_train, y_train_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

    print("TESTING RESULTS: \n===============================")
    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True, zero_division=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_test_pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")

In [17]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

def ensemble_methods(method, n_estimators = 50):
    model = DecisionTreeClassifier()
    if method == 'BaggingClassifier':
        clf = BaggingClassifier(base_estimator=model, 
                                n_estimators=n_estimators, 
                                random_state=42)
    elif method == 'RandomForestClassifier':
        clf = RandomForestClassifier(n_estimators=n_estimators, 
                                    random_state=42)
    elif method == 'AdaBoostClassifier':
        clf = AdaBoostClassifier(n_estimators=n_estimators)
    else:
        return
    return clf


In [22]:
def get_metrics(dataset, label, model, test_size_arr):
    accuracy_arr = []
    precision_arr = []
    recall_arr = []
    f1_arr = []
    for test_size in test_size_arr:
        # Split train and test data
        X_train, X_test, y_train, y_test= get_splitted_train_test(dataset, label, test_size)

        # Encode categorical variables
        X_train, X_test = get_encoded_X_data(X_train, X_test)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)  
        
        # Save metrics
        accuracy_arr.append(sklearn.metrics.accuracy_score(y_test, y_pred))
        precision_arr.append(sklearn.metrics.precision_score(y_test, y_pred, average='weighted', zero_division=1))
        recall_arr.append(sklearn.metrics.recall_score(y_test, y_pred, average='weighted', zero_division=1))
        f1_arr.append(sklearn.metrics.f1_score(y_test, y_pred, average='weighted'))

    return accuracy_arr, precision_arr, recall_arr, f1_arr

In [25]:
def display_list(name, arr):
    print(name)
    for i in range(len(arr)):
        print(f"{arr[i]}")
    return

Census Income dataset

In [27]:
DATASET_PATH = os.path.join('D:\projects\decision_tree_classification\datasets','adult.data')
label = "income" # label
dataset = pd.read_csv(DATASET_PATH, sep=", ")

C:\Users\lerus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Grades dataset

In [44]:
DATASET_PATH = os.path.join('D:\projects\decision_tree_classification\datasets','grades.csv')
label = "GRADE"
dataset = pd.read_csv(DATASET_PATH)

In [46]:
# CRITERION='gini'
# CRITERION='entropy'
TEST_SIZE=0.3
# TEST_SIZE_ARR = [0.4, 0.3, 0.2, 0.1]
TEST_SIZE_ARR = [0.3]
methods = ['BaggingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier']
# N_ESTIMATORS=50

In [48]:
for METHOD in methods:
    print(f'Dataset = {DATASET_PATH}')
    print(f'METHOD = {METHOD}')
    print('N_ESTIMATORS,ACCURACY,PRECISION,RECALL,F1-SCORE')
    for N_ESTIMATORS in range(50,110,10):  
        classifier = ensemble_methods(method = METHOD, n_estimators = N_ESTIMATORS)
        accuracy_arr, precision_arr, recall_arr, f1_arr = get_metrics(dataset, label, model = classifier, test_size_arr = TEST_SIZE_ARR)
        
        print(f'{N_ESTIMATORS},{accuracy_arr[0]},{precision_arr[0]},{recall_arr[0]},{f1_arr[0]}')

Dataset = D:\projects\decision_tree_classification\datasets\grades.csv
METHOD = BaggingClassifier
N_ESTIMATORS,ACCURACY,PRECISION,RECALL,F1-SCORE
50,0.3181818181818182,0.4777777777777778,0.3181818181818182,0.27171717171717175
60,0.3181818181818182,0.4777777777777778,0.3181818181818182,0.27171717171717175
70,0.3181818181818182,0.4777777777777778,0.3181818181818182,0.27171717171717175
80,0.3181818181818182,0.4777777777777778,0.3181818181818182,0.27171717171717175
90,0.3181818181818182,0.4777777777777778,0.3181818181818182,0.27171717171717175
100,0.3181818181818182,0.4777777777777778,0.3181818181818182,0.27171717171717175
Dataset = D:\projects\decision_tree_classification\datasets\grades.csv
METHOD = RandomForestClassifier
N_ESTIMATORS,ACCURACY,PRECISION,RECALL,F1-SCORE
50,0.2727272727272727,0.3162878787878788,0.2727272727272727,0.24300144300144302
60,0.3181818181818182,0.37077922077922076,0.3181818181818182,0.22742030696576154
70,0.3181818181818182,0.3996212121212121,0.3181818181818182,0

In [29]:
classifier = ensemble_methods(method = METHOD, n_estimators = N_ESTIMATORS)
accuracy_arr, precision_arr, recall_arr, f1_arr = get_metrics(dataset, label, model = classifier, test_size_arr = TEST_SIZE_ARR)

In [30]:
display_list("test_size", TEST_SIZE_ARR)
display_list("accuracy", accuracy_arr)
display_list("precision", precision_arr)
display_list("recall", recall_arr)
display_list("f1-score", f1_arr)

test_size
0.4
0.3
0.2
0.1
accuracy
0.8272552783109405
0.8394922714709796
0.8367879625364655
0.8345102855388394
precision
0.8186485278283226
0.8336492014378727
0.8328129532175675
0.8318479641756297
recall
0.8272552783109405
0.8394922714709796
0.8367879625364655
0.8345102855388394
f1-score
0.8208442574207844
0.8356292377115739
0.8344260292650884
0.833021920430877
